In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Краткое описание
Ваша задача разработать систему, способную разрешать неоднозначность, возникающую в употреблении омонимичных форм.

In [2]:
data = pd.read_csv("train.csv", sep='\t')
data.head(480)

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,NaN,замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,NaN,"шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,NaN,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,NaN,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,NaN,великий князь литовский гедимин после успешной...
5,6,замок,1,NaN,NaN,его без боя . в начале xviii века высокий замо...
6,7,замок,1,NaN,NaN,", колодец , хозяйственные помещения ( склады п..."
7,8,замок,1,NaN,NaN,— сын гедимина трокский князь кейстут . около ...
8,9,замок,1,NaN,NaN,одним из кэмпбеллов — отдал замок в обмен на в...
9,10,замок,1,NaN,NaN,замок сталкер замок был построен в г . и перв...


Вам дается набор "главных слов". Или слов, которые имеют несколько возможных смыслов в тексте.

Например, слово лук может встречаться в значении оружие или в значении овощ.

Ваша задача сопоставить одинаковые метки тем контекстам, в которых "главное слово" встречается в одном и том же значении. Важно учесть, что предполагается, что число возможных смыслов у "главного слова" заранее неизвестно. Таким образом это фактически задача кластеризации с заранее неизвестным числом классов.

Также предполагается, что система будет применяться к "главным словам", которых нет в вашей обучающей выборке.

В текущем варианте задания ваша задача построить и протестировать систему на датасете wiki-wiki, собранном из статей википедии.

Это очень маленький датасет, в нем всего 4 "главных слова":

In [3]:
data.word.unique()

array(['замок', 'лук', 'суда', 'бор'], dtype=object)

Т.к. это тренировочный датасет, правильные метки для каждого контекста известны и можно посмотреть некоторую статистику по ним.

#### Сколько контекстов приходится на каждое слово?

In [4]:
data.loc[:,['word', 'gold_sense_id', 'context']].groupby(['word']).count()['context'].sort_values(ascending=False)

word
замок    138
суда     135
лук      110
бор       56
Name: context, dtype: int64

#### Cколько контекстов приходится на одно смысловое значение слова?

In [5]:
data.loc[:,['word', 'gold_sense_id', 'context']].groupby(['word', 'gold_sense_id'])['context'].count()

word   gold_sense_id
бор    1                 14
       2                 42
замок  1                100
       2                 38
лук    1                 65
       2                 45
суда   1                100
       2                 35
Name: context, dtype: int64

### Важное замечание

Как можно заметить, в этом датасете ровно по 2 смысла у каждого слова.

Таким образом, разрешается в качестве k у KMeans взять значение 2.

### Про кластеризацию

In [6]:
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
import pandas as pd

In [7]:
# запускаем кластеризацию отдельно для каждого набора контекстов, соответствующих своему "главному слову"
km = KMeans(n_clusters = 2)

In [8]:
import wget

udpipe_url = 'http://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(udpipe_url)

In [9]:
from ufal.udpipe import Model, Pipeline

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    model = Model.load(modelfile)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    processed = pipeline.process(text) # обрабатываем текст, получаем результат в формате conllu
    output = [l for l in processed.split('\n') if not l.startswith('#')] # пропускаем строки со служебной информацией
    tagged = [w.split('\t')[2].lower() + '_' + w.split('\t')[3] for w in output if w] # извлекаем из обработанного текста лемму и тэг
    tagged_propn = []
    propn  = []
    for t in tagged:
        if t.endswith('PROPN'):
            if propn:
                propn.append(t)
            else:
                propn = [t]
        else:
            if len(propn) > 1:
                name = '::'.join([x.split('_')[0] for x in propn]) + '_PROPN'
                tagged_propn.append(name)
            elif len(propn) == 1:
                tagged_propn.append(propn[0])
            tagged_propn.append(t)
            propn = []
    return tagged_propn


In [11]:
processed_ud = []
for i in range(len(list(data['context']))):
    text = data['context'][i]
    processed_ud.append(tag_ud(text=text, modelfile=modelfile))
print(processed_ud[:30])

[['замок_NOUN', 'владимир_NOUN', 'мономах_NOUN', 'в_ADP', 'любекий_NOUN', '._PUNCT', 'многочисленный_ADJ', 'укреплять_VERB', 'монастырь_NOUN', 'также_ADV', 'не_PART', 'являться_VERB', 'замок_NOUN', 'как_SCONJ', 'таковой_ADJ', '—_PUNCT', 'это_PRON', 'быть_AUX', 'крепость_NOUN', '._PUNCT', 'ранний_ADJ', 'европейский_ADJ', 'замок_NOUN', 'строить_VERB', 'преимущественно_ADV', 'из_ADP', 'дерево_NOUN', 'они_PRON', 'опоясываться_VERB', 'деревянный_ADJ', 'ограда_NOUN', '—_PUNCT', 'палисад_NOUN', 'уже_ADV', 'тогда_ADV', 'вокруг_ADP', 'замок_NOUN', 'стать_VERB', 'появляться_VERB', 'ров_NOUN', '._PUNCT', 'пример_NOUN', 'такой_DET', 'замок_NOUN', 'мочь_VERB', 'служить_VERB', 'вышгородский_ADJ', 'замок_NOUN', 'киевский_ADJ', 'князь_NOUN', '._PUNCT', 'каменный_ADJ', 'замковый_ADJ', 'строительство_NOUN', 'распространиться_VERB', 'в_ADP', 'западный_ADJ', 'и_CCONJ', 'центральный_ADJ', 'европа_NOUN', 'лишь_PART', 'к_ADP', 'xii_NUM', 'век_NOUN', '._PUNCT', 'главный_ADJ', 'часть_NOUN', 'средневековый_ADJ'

In [12]:
#Качаю модель на основе НКРЯ
import gensim

model_url = 'http://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz'
modelfile = wget.download(model_url)
m = 'ruscorpora_upos_skipgram_300_5_2018.vec.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.Word2Vec.load(m)

In [28]:
#нормализация модели, чтобы она занимала меньше места
model.init_sims(replace=True)

In [13]:
import re
expr = r'[^\w]'
parser = re.compile(expr)
data['context'] = [parser.sub(r' ', word) for word in data['context']]

In [14]:
#удалил все слова не из модели
vocab = model.vocab
contexts = data['context'].values

contexts_clean = []
for context in contexts:
    context_clean = []
    words = tag_ud(text=context, modelfile='udpipe_syntagrus.model')
    for word in words:
        if word in vocab:
            context_clean.append(word)
    contexts_clean.append(context_clean)

data['context'] = contexts_clean

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
tfidf_matrix =  tf.fit_transform(" ".join(context) for context in data['context'].values)

In [26]:
contexts_vec = []
for doc, context in enumerate(data['context'].values): 
    words_vec = []
    tfidfs =[]
    feature_names = tf.get_feature_names()
    feature_indices = tfidf_matrix[doc,:].nonzero()[1]
    #векторизуем слова
    for word in list(set(context)):
        try:
            word_vec = model.get_vector(word)
            words_vec.append(word_vec)
        except:
            continue
            
            
    #получим tfidf 
    for word in list(set(context)):
        try:
            feature_index = feature_names.index(word.lower())
            tfidf = tfidf_matrix[doc, feature_index]
        except:
            tfidf = 0
        tfidfs.append(tfidf)
#    print(doc)

    context_vec = np.dot(tfidfs, words_vec)
    contexts_vec.append(context_vec)

In [27]:
#наканецта
for main_word in data.word.unique():
    indixes = list(data[data.word == main_word].index)
    data['predict_sense_id'][indixes] = km.fit_predict(np.array(contexts_vec)[indixes])
    score = adjusted_rand_score(data['predict_sense_id'][indixes], data['gold_sense_id'][indixes])
    print(main_word, score)

/home/nullkatar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


замок 0.23824273901619683
лук 0.9279187206925574
суда 0.33798134638470767
бор 1.0


In [18]:
np.shape(context_clean)

(22,)

In [31]:

for main_word in data.word.unique():
    print(data[data.word == main_word].index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            128, 129, 130, 131, 132, 133, 134, 135, 136, 137],
           dtype='int64', length=138)
Int64Index([138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
            ...
            238, 239, 240, 241, 242, 243, 244, 245, 246, 247],
           dtype='int64', length=110)
Int64Index([248, 249, 250, 251, 252, 253, 254, 255, 256, 257,
            ...
            373, 374, 375, 376, 377, 378, 379, 380, 381, 382],
           dtype='int64', length=135)
Int64Index([383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395,
            396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408,
            409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421,
            422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434,
            435, 436, 437, 438],
           dtype='int64')


In [30]:
np.shape(context_vec[1])

()

Что нужно сделать, чтобы посчитать метрику ARI?

In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("../models/model_big_one.vec", binary=False)